In [1]:
import plotly.graph_objects as go
import kaleido
import numpy as np

In [2]:
# Function to calculate APR
def calculate_apr(ratio_normal_staked, num_minipools, no_commission_dict, nrbms_commission):
    if num_minipools == 1:
        return 100  # 32 ETH validator always stays at 100% APR

    no_commission = no_commission_dict[num_minipools]
    reward_from_reth_holders = (num_minipools * (100 - (100/num_minipools))/(100)*no_commission)
    
    # Avoiding division by zero in case ratio_normal_staked is 1
    if ratio_normal_staked == 1:
        reward_from_nrbms = 0
    else:
        reward_from_nrbms = (nrbms_commission/(1-ratio_normal_staked)) - nrbms_commission

    return 100 + reward_from_reth_holders + reward_from_nrbms


In [3]:


nrbms_commission = 5
no_commission_dict = {16: 12, 8: 13, 4: 14, 2: 15}

num_minipools_options = [2, 4, 8, 16]
ratio_normal_staked_values = np.linspace(0, 0.97, 50)

# Create the figure
fig = go.Figure()

# Add traces for each number of minipools
for num_minipools in num_minipools_options:
    apr_values = [calculate_apr(ratio, num_minipools, no_commission_dict, nrbms_commission) for ratio in ratio_normal_staked_values]
    fig.add_trace(go.Scatter(
        x=ratio_normal_staked_values,
        y=apr_values,
        mode='lines+markers',
        name=f'{num_minipools} minipools',
        line=dict(width=2)
    ))

# Add horizontal lines for Lido CSM and Solo Staking
fig.add_trace(go.Scatter(
    x=[0, 1], y=[153, 153], mode='lines', name='Lido CSM at 7.5%', line=dict(color='orange', width=2, dash='dash')))
fig.add_trace(go.Scatter(
    x=[0, 1], y=[100, 100], mode='lines', name='Solo Staking', line=dict(color='green', width=2, dash='dash')))

# Update the layout for better visualization
fig.update_layout(
    title=f'ETH Staking Rewards with Varied Staking Ratios - {nrbms_commission}% NRBM Commission',
    xaxis_title='Ratio of NRBM staked ETH',
    yaxis_title='Total APR (%)',
    legend=dict(x=0.5, y=0.97, orientation='h'),
    width=800,
    height=600,
    paper_bgcolor='rgba(255,255,255,1)',
    plot_bgcolor='rgba(255,255,255,1)',
    xaxis=dict(gridcolor='rgba(200,200,200,0.5)'),
    yaxis=dict(gridcolor='rgba(200,200,200,0.5)')
)

# Show the updated plot
fig.show()

In [7]:
# Save the plot as a JPEG file
fig.write_image("no_rewards_varied_staking_ratio.jpg")

In [6]:
# Save the plot as an interactive HTML file
fig.write_html("eth_staking_rewards_interactive.html")